# Approach 2 - method 2 - step2 (Extract Features)
- Precisely crop the anomaly from the detections by syncing the subtrace before and after the anomaly w.r.t ref_samples
- to keep the lenght of feature vector same, we pad the features with trailing zeros to get length of 500 (max length of detection)
- The feature extraction is the dependent on the corresponding normal behaviour subtrace
- We tested this approach across all applications


In [ ]:
import json
import os
import sys
sys.path.append('../')  ### to detect libraries in the parent directory
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import defaultdict
from libraries.utils import *
from libraries.exeint import exeInt
import plotly.express as px
from statistics import mode

# ############ configuration - trace ################
# ############################################


CODE = 'theft_protection'       ### application (code)       ###  'theft_protection', 'mamba2', 'lora_ducy'
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 4                     ### format of data collection
WINDOW = 500                 ### window size for subsequence

base_dir = '../../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)


################# configuration - diag ################
IS_VAR_WINDOW = False             ### True, False; wether to use variable window size or not

#####################################################


ref_samples_basepath = os.path.join(normalbase_path, f'diag_refsamples{WINDOW}')
ref_var_samples_basepath = os.path.join(normalbase_path, 'diag_var_refsamples')
diag_subseq_basepath = os.path.join(faultybase_path, 'diag_subseq')
subseq_label_basepath = os.path.join(diag_subseq_basepath, 'subseq_labels')
test_labels_basepath = os.path.join(faultybase_path, 'labels')


print('ref_samples_path:\n', ref_samples_basepath)
print('ref_var_samples_path:\n', ref_var_samples_basepath)
print('diag_subseq_path:\n', diag_subseq_basepath)

######### get paths #######################
ref_samples_path = [os.path.join(ref_samples_basepath, x) for x in os.listdir(ref_samples_basepath)]
# ref_var_samples_path = [os.path.join(ref_var_samples_basepath, x) for x in os.listdir(ref_var_samples_basepath)]   

train_varlist_path = os.listdir(normalbase_path)
train_varlist_path = [os.path.join(normalbase_path, x) for x in train_varlist_path if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

test_subseq_path = [os.path.join(diag_subseq_basepath, x) for x in os.listdir(diag_subseq_basepath)]
test_labels_path = [os.path.join(subseq_label_basepath, x) for x in os.listdir(subseq_label_basepath)]
eval_labels_path = [os.path.join(test_labels_basepath, x) for x in os.listdir(test_labels_basepath)]


# ### remove.Ds_store from all lists
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]
ref_samples_path = [x for x in ref_samples_path if '.DS_Store' not in x]
# ref_var_samples_path = [x for x in ref_var_samples_path if '.DS_Store' not in x]
test_subseq_path = [x for x in test_subseq_path if '.DS_Store' not in x if '.json' in x]
test_labels_path = [x for x in test_labels_path if '.DS_Store' not in x]
eval_labels_path = [x for x in eval_labels_path if '.DS_Store' not in x]

varlist_path.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

if IS_VAR_WINDOW:
    # train_data_path = ref_var_samples_path
    raise ValueError('Ref samples for variable window missing')
else:
    train_data_path = ref_samples_path

test_data_path = test_subseq_path

# print('train_data:', train_data_path)
print(len(train_data_path))
# print('test_data:\n', test_data_path)
print(len(test_data_path))
print('test_labels:\n', test_labels_path)
print('eval_labels:\n', eval_labels_path)



In [ ]:
#### Load diag_sepAP2_m2

path_diag_sepAP2_m2 = os.path.join(faultybase_path, 'diag_sepAP2_m2')
path_labels_diag_sepAP2_m2 = os.path.join(path_diag_sepAP2_m2, 'subseq_labels')
print('path_diag_sepAP2_m2:', path_diag_sepAP2_m2)
print('path_labels_diag_sepAP2_m2:', path_labels_diag_sepAP2_m2)

files_sepap2 = os.listdir(path_diag_sepAP2_m2)
files_sepap2 = [os.path.join(path_diag_sepAP2_m2, x) for x in files_sepap2 if '.DS_Store' not in x]    ### remove .DS_Store
files_sepap2 = [x for x in files_sepap2 if os.path.isfile(x)]

labels_sepap2 = os.listdir(path_labels_diag_sepAP2_m2)
labels_sepap2 = [os.path.join(path_labels_diag_sepAP2_m2, x) for x in labels_sepap2 if '.DS_Store' not in x]    ### remove .DS_Store
labels_sepap2 = [x for x in labels_sepap2 if os.path.isfile(x)]


In [ ]:
##### Get file names for desired class
### class = 0, 1, 2; -1 for all classes
CLASS = 2

#### Load labels for diag_sepAP2_m2
labels_dict = read_json(labels_sepap2[0])
print('labels_dict:', labels_dict)


key_names = list(labels_dict.keys())
# print('key_names:', key_names)

sel_ap2_files = []
sel_ap2_classes = []
for km in key_names:
    print('km:', km)
    _class = labels_dict[km][0]
    print('class:', _class)

    if _class == CLASS or CLASS == -1:
        _file_name = os.path.join(path_diag_sepAP2_m2, km+'.json')
        print('file:', _file_name)
        ############# check if file exists or not
        if os.path.isfile(_file_name):
            print('file exists')
            sel_ap2_files.append(_file_name)
            sel_ap2_classes.append(_class)
        else:
           raise ValueError('File not found:', _file_name)



In [ ]:
len(sel_ap2_files)

In [ ]:
if CODE == 'theft_protection':
        EVENT_MEDIAN = 22
        WINDOW = round(2 * EVENT_MEDIAN)    ### round(2 * median event range)
        SLIDING_WINDOW = round(EVENT_MEDIAN * 1.5)    ### round( 0,4 * WINDOW)
        # SLIDING_WINDOW = round(WINDOW - 25)   
        print('WINDOW:', WINDOW)
        print('SLIDING_WINDOW:', SLIDING_WINDOW) 
elif CODE == 'mamba2':
    EVENT_MEDIAN = 57
    WINDOW = round(2 * EVENT_MEDIAN)  ### round(1,8 * median event range)
    SLIDING_WINDOW = round(EVENT_MEDIAN * 1.5)    ### round( 0,7 * WINDOW)
    # SLIDING_WINDOW = round(WINDOW - 20)    
    print('WINDOW:', WINDOW)
    print('SLIDING_WINDOW:', SLIDING_WINDOW)

#### Load files
sel_subseq = []
i= 0
for sf in sel_ap2_files:
    print('sf:', sf)
    det_subseq = read_json(sf)
    print('subseq:', det_subseq)
    sel_subseq.append(det_subseq)

    print('classes:', sel_ap2_classes[i])

    '''
    TODO:
    - get first half subseq
    - get second half subseq
    - get ref_samples for each half subseq
    - compare subseq to obtain add/missing events
    '''
    # print('SLIDING_WINDOW:', SLIDING_WINDOW)

    # #### load corresponding test file (under the assumption that we will have subseq of 500 events from the node when a fault is detected. It can be bigger or smaller depending upon the selected buffer size)
    # test_file_name = sf.split('/')[-1]
    # test_file_name = test_file_name.split('.')[0]
    # detection_indices = test_file_name.split('_')[-1]
    # test_file_name = '_'.join(test_file_name.split('_')[:-1])
    # test_file_path = os.path.join(faultybase_path, test_file_name)
    # print('test_file_name:', test_file_name)
    # print('test_file_path:', test_file_path)
    # print('detection_indices:', detection_indices)

    # #### load test file
    # test_subseq = read_json(test_file_path)
    # print('test_subseq:', test_subseq)

    # det_start = int(detection_indices.split('-')[0])
    # det_end = int(detection_indices.split('-')[1])
    # print('det_start:', det_start)

    ### verify if indices are correct
    # _det_subseq = test_subseq[det_start:det_end]
    # print('_det_subseq:', _det_subseq)
    
    # #### get first half subseq
    # fh_start_ind = det_start - int(SLIDING_WINDOW)
    # fh_end_ind = det_start
    # print('fh_start_ind:', fh_start_ind)
    # print('fh_end_ind:', fh_end_ind)
    # fh_subseq = test_subseq[fh_start_ind:fh_end_ind]
    # print('fh_subseq:', fh_subseq)



    # #### get second half subseq
    # sh_start_ind = det_end
    # sh_end_ind = det_end + int(SLIDING_WINDOW)
    # print('sh_start_ind:', sh_start_ind)
    # print('sh_end_ind:', sh_end_ind)

    i += 1

    # break